# auth-proxyのメタデータを登録する

---

IdP-proxyにCoursewareHub(auth-proxy)のメタデータを追加する

## 概要

このNotebookでは「211-学認連携の設定を行う-IdP-proxyを利用する.ipynb」で設定したauth-proxyコンテナからメタデータを取得してIdP-proxyへの登録を行います。

IdP-proxyとCoursewareHubの構築担当者が異なる場合は、IdP-proxy構築担当者がCoursewareHub構築担当者からの依頼を受けてこのNotebookを実行してください。

メタデータ登録の際はIdP-proxyからネットワークを通して直接auth-proxyのメタデータを取得します。そのため事前の準備としてIdP-proxyからauth-proxyにアクセスできるようにファイアウォールなどの設定を変更しておいてください。

## 操作対象の設定

操作対象となるAnsibleのグループ名を指定します。

### グループ名

IdP-proxyの構築環境のグループ名を指定します。

ノードを作成した時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

グループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_auth = 'IdPproxy'

target_auth = 

### チェック

指定されたグループに属するノードがAnsibleによって操作できることを確認します。

グループ名に属するノードがアクセスできることを確認します。

In [ ]:
!ansible {target_auth} -m ping

管理者権限での操作ができることを確認します。

In [ ]:
!ansible {target_auth} -b -a 'whoami'

## メタデータの追加

CoursewareHub(auth-proxy)のメタデータをIdP-proxyに登録します。

登録するCoursewareHubのホスト名(FQDN)を指定してください。

In [ ]:
# (例)
# master_fqdn = 'hub.example.org'

master_fqdn = 

IdP-proxyから指定されたCoursewareHubのメタデータが取得できること確認します。

In [ ]:
!ansible {target_auth} -b -a 'chdir=/srv/idp-proxy docker compose exec idp-proxy \
    curl -sfI https://{master_fqdn}/simplesaml/module.php/saml/sp/metadata.php/default-sp'

メタデータをIdP-proxyに登録します。IdP-proxyコンテナの`add_auth_proxy.sh`を実行することでIdP-proxyコンテナの`/var/www/simplesamlphp/metadata/xml/auth-proxies.xml`にメタデータが追加されます。

In [ ]:
!ansible {target_auth} -b -a 'chdir=/srv/idp-proxy docker compose exec idp-proxy \
    /usr/local/sbin/add_auth_proxy.sh {master_fqdn}'